In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import xlrd
import math
import time

In [22]:
# Step 1: Read in Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/mnist',one_hot=True)
X_train,Y_train = mnist.train.next_batch(60000)
X_test,Y_test = mnist.test.next_batch(10000)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [23]:
num_train=60000
num_test =10000

In [24]:
def get_next_batch_RAM(X,Y,batch_size,batch_number):
    num_classes = 10
    # Indices to slice data into batches
    start = batch_number*batch_size
    end = start + batch_size
    batch_X = X[start:end,:]
    batch_Y = Y[start:end,:]
    return batch_X,batch_Y

In [25]:
# Defing Model
# Define parameters 
tf.reset_default_graph()
learning_rate = 0.1
batch_size = 128
n_epochs = 100
lamda = 0.0001
graph1 = tf.Graph()
with graph1.as_default():
    #Step 2: 
    X = tf.placeholder(tf.float32,[batch_size,784],name='X_placeholder')
    Y = tf.placeholder(tf.int32,[batch_size,10],name='Y')
    #Step 3:
    weights = tf.Variable(tf.random_normal(shape=[784,10],stddev=0.01),name='weights') 
    bias = tf.Variable(tf.zeros([1,10]),name='bias')

    #Step 4:
    logits = tf.matmul(X,weights) + bias
    
    #Step 5: 
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y,logits=logits,name='loss')
    loss = tf.reduce_mean(entropy) + lamda*tf.norm(weights,ord=2)
    # Step 6:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)    

In [26]:

tf.reset_default_graph()
# Parameters and variables

# Number of batches for test and train set
n_batches_train = int(math.floor(num_train/batch_size))
n_batches_test = int(math.floor(num_test/batch_size)) 
#print(n_batches_train)
# Run Session
with tf.Session(graph=graph1) as sess:
    #writer = tf.summary.FileWriter('./graphs/assignment1_b',sess.graph)
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        total_loss = 0
        for j in range(n_batches_train):
            batch_number = j
            X_batch,Y_batch = get_next_batch_RAM(X_train,Y_train,batch_size,batch_number)
            _,loss_batch = sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss +=loss_batch
            # Prints one hash after 10 batches
            if (j%10==0):
                print('#', end = "")
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches_train))    
    print('Total_time:{0} seconds',format(time.time()-start_time))
    
# Test The model
    n_batches = math.floor((num_test)/batch_size)
    total_correct_preds = 0
    for i in range(n_batches_test):
        #print(i,end='')
        batch_number = i
        X_batch,Y_batch = get_next_batch_RAM(X_test,Y_test,batch_size,batch_number)
        #loss_batch,logits_batch = sess.run([loss,logits],feed_dict={X:X_batch,Y:Y_batch})
        probs = tf.nn.softmax(logits)
        correct_preds = tf.equal(tf.argmax(probs,1),tf.argmax(Y_batch,1))
        n_correct_preds = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
        total_correct_preds += sess.run(n_correct_preds,feed_dict={X:X_batch,Y:Y_batch})
    print('Accuracy:{0}'.format(total_correct_preds/num_test))  
    #print('Loss{0}'.format(loss_total)) 
#writer.close()           

###############################################Average loss epoch 0:0.5744087563620673
###############################################Average loss epoch 1:0.37557274864142776
###############################################Average loss epoch 2:0.3438213505487666
###############################################Average loss epoch 3:0.3272504346747684
###############################################Average loss epoch 4:0.3165420026032843
###############################################Average loss epoch 5:0.30885641668469477
###############################################Average loss epoch 6:0.3029764636268473
###############################################Average loss epoch 7:0.2982781028263589
###############################################Average loss epoch 8:0.294403268613367
###############################################Average loss epoch 9:0.2911297579924775
###############################################Average loss epoch 10:0.2883114574684037
#########################################

In [17]:
# Step 1: Read in Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/data/mnist',one_hot=False)
X_train,Y_train = mnist.train.next_batch(60000)
X_test,Y_test = mnist.test.next_batch(10000)

Extracting /data/mnist\train-images-idx3-ubyte.gz
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [19]:
from sklearn import linear_model
from sklearn import metrics
LR = linear_model.LogisticRegression(max_iter=100)
LR.fit(X_train,Y_train)
y_pred = LR.predict(X_test)
accuracy = metrics.accuracy_score(Y_test, y_pred, sample_weight=None)
print(accuracy)

0.92
